<a href="https://colab.research.google.com/github/ApoorvaNagarajan/Image-Filter-Kernels/blob/master/assignment4/assignment4_iteration3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries and modules**

**Idea behind the network**

- Has three improvements over iteration2
- Improvements
  - Add dropout layers to avoid overfitting
  - As we have room to increase number of parameters, increase number of channels to 16 on some layers. This will help us increase accuracy
  - Increase number of epochs to see if the network will learn better with more epochs

**Observations : Filled after running the network below. (Have put it in the top so that this is not missed out)**

- Parameters : 14,550
- Epochs : 20
- val_acc : 0.9944 (at 10th epoch)
- Time taken for each epoch : 120us
- The target parameters and accuracy both have been met
- As and when the number of iterations increases beyond 10, we can observe the issue of overfitting again. we can try to experiment with the dropout rates further
- Since this is only third iteration, in the next iteration, I will try to improve accuracy and parameters further

In [2]:
# Import all the required python libraries
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.datasets import mnist

Using TensorFlow backend.


### Load pre-shuffled MNIST data into train and test sets

In [3]:
# downloads the MNIST dataset and splits it into test and train.
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# X_train and X_test are reshaped from 3 dimensions to 4 dimensions
# 1st dim : num images
# 2nd and 3rd dim : Width and height of eaach of the images
# 4th dim : number of channels in each of the image. We set this to 1 as the
#           our dataset has only gray images
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)


# Normalizing the image
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

11493376/11490434 [==============================] - 2s 0us/step


In [7]:
from keras.layers import Activation
model = Sequential()

 
model.add(Convolution2D(10, 3, 3, activation='relu', input_shape=(28,28,1))) # receptive field: 3x3, output dim: 26x26x10
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Convolution2D(16, 3, 3, activation='relu')) # receptive field: 5x5, output dim: 24x24x16
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Convolution2D(20, 3, 3, activation='relu')) # receptive field: 7x7, output dim: 22x22x20
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(MaxPooling2D(pool_size=(2, 2))) # receptive field: 14x14, output dim: 11x11x20
model.add(Convolution2D(10, 1, 1, activation='relu')) # receptive field: 14x14, output dim: 11x11x10
model.add(BatchNormalization())

model.add(Convolution2D(16, 3, 3, activation='relu')) # receptive field: 16x16, output dim: 9x9x16
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Convolution2D(20, 3, 3, activation='relu')) # receptive field: 18x18, output dim: 7x7x20
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Convolution2D(10, 1, activation='relu')) # receptive field: 18x18, output dim: 7x7x10
model.add(BatchNormalization())
model.add(Convolution2D(10, 7)) # receptive field: 25x25, output dim: 1x1x10
model.add(Flatten())
model.add(Activation('softmax'))


model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (3, 3), activation="relu", input_shape=(28, 28, 1...)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (1, 1), activation="relu")`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 10)        100       
_________________________________________________________________
batch_normalization_8 (Batch (None, 26, 26, 10)        40        
_________________________________________________________________
dropout_6 (Dropout)          (None, 26, 26, 10)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 16)        1456      
_________________________________________________________________
batch_normalization_9 (Batch (None, 24, 24, 16)        64        
_________________________________________________________________
dropout_7 (Dropout)          (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 22, 22, 20)        2900      
__________

In [0]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
  return round(0.005 * 1/(1 + 0.25 * epoch), 10)

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.005), metrics=['accuracy'])


In [22]:
model.fit(X_train, Y_train, batch_size=128, epochs=20, verbose=1, validation_data=(X_test, Y_test), callbacks=[LearningRateScheduler(scheduler, verbose=1)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20

Epoch 00001: LearningRateScheduler setting learning rate to 0.005.
60000/60000 [==============================] - 11s 175us/step - loss: 0.0237 - acc: 0.9924 - val_loss: 0.0240 - val_acc: 0.9925
Epoch 2/20

Epoch 00002: LearningRateScheduler setting learning rate to 0.004.
60000/60000 [==============================] - 7s 115us/step - loss: 0.0182 - acc: 0.9938 - val_loss: 0.0273 - val_acc: 0.9910
Epoch 3/20

Epoch 00003: LearningRateScheduler setting learning rate to 0.0033333333.
60000/60000 [==============================] - 7s 112us/step - loss: 0.0164 - acc: 0.9945 - val_loss: 0.0241 - val_acc: 0.9929
Epoch 4/20

Epoch 00004: LearningRateScheduler setting learning rate to 0.0028571429.
60000/60000 [==============================] - 7s 112us/step - loss: 0.0152 - acc: 0.9948 - val_loss: 0.0235 - val_acc: 0.9926
Epoch 5/20

Epoch 00005: LearningRateScheduler setting learning rate to 0.0025.
60000/60000 [==================

In [23]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

[0.021978519912086086, 0.9933]
